# Reading Tabular Data into DataFrames

## Getting data from CDAWeb using `sunpy`

(Based on https://docs.sunpy.org/en/stable/generated/gallery/acquiring_data/search_cdaweb.html#sphx-glr-generated-gallery-acquiring-data-search-cdaweb-py)

The Coordinated Data Analysis Web (CDAWeb) stores data from from current and past space physics missions, and is full of heliospheric insitu datasets.

CDAWeb stores data from from current and past space physics missions, and is full of heliospheric insitu datasets.

First, we need to install `sunpy` and a couple of other dependencies. In most Python environments the command would be `pip install <module>`, but we need to modify that slightly for it to work correctly in a Jupyter Notebook.

In [ ]:
# Install a pip package in the current Jupyter kernel
#
# While sunpy is what we're looking for, trying to run the code below
# without the rest of the packages results in errors
#
# Matplotlib is needed when running in Binder - should already be
# present for Anaconda installs
import sys
!{sys.executable} -m pip install sunpy
# !{sys.executable} -m pip install sunpy drms cdflib zeep h5netcdf matplotlib


Now, we can import the modules we need from `sunpy`:

In [ ]:
from sunpy.net import Fido
from sunpy.net import attrs as a
from sunpy.timeseries import TimeSeries

`sunpy.net.Fido` is the primary interface to search for and download data and will automatically search CDAWeb when the `cdaweb.Dataset` attribute is provided to the search. To lookup the different dataset IDs available, you can use the form at https://cdaweb.gsfc.nasa.gov/index.html/. Here, we're getting Solar Orbiter Level 2 Magnetometer Data in RTN Coordinates in Normal Mode.

In [ ]:
date_range = a.Time('2021/07/01', '2021/07/08')
dataset = a.cdaweb.Dataset('SOLO_L2_MAG-RTN-NORMAL-1-MINUTE')
result = Fido.search(date_range, dataset)

Let's inspect the results. We can see that there's seven files, one for each day within the query.

In [ ]:
print(result)

We have something that looks a bit like a list of files from different providers. In our particular case, there is only one provider, so we can get the files from that:

In [ ]:
print(result[0])

But using a higher index results in an error - there are no files from any other providers.

In [ ]:
print(result[1])

We can look at the individual files in the set:

In [ ]:
print(result[0,0])

In [ ]:
print(result[0,1])

We can use a slice to view a subset of the files:

In [ ]:
print(result[0,0:2])

We can use `Fido.fetch()` to download the contents of the specified files:

In [ ]:
downloaded_files = Fido.fetch(result[0, 0:2])

We can then concatenate the contents of those files in a more readily usable form using `TimeSeries`

In [ ]:
solo_mag = TimeSeries(downloaded_files, concatenate=True)

Looking at the type of `solo_mag` we can see that it is of a type which is defined within sunpy. We can use `help()` to find out a bit more about it.

In [ ]:
print(type(solo_mag))

In [ ]:
import sunpy.timeseries
help(sunpy.timeseries.timeseriesbase.GenericTimeSeries)

One of the methods on `GenericTimeSeries` is `to_dataframe()`, which returns the data contained in the timeseries as a _Pandas dataframe_.

## Use the Pandas library to do statistics on tabular data.

- [Pandas](https://pandas.pydata.org/) is a widely-used Python library for statistics, particularly on tabular data.
- Borrows many features from R’s dataframes.
  - A 2-dimensional table whose columns have names and potentially have different data types.

In [ ]:
solo_mag_data = solo_mag.to_dataframe()
type(solo_mag_data)

In [ ]:
print(solo_mag_data)

- The columns in a dataframe are the observed variables, and the rows are the observations.
- Pandas uses backslash `\` to show wrapped lines when output is too wide to fit the screen.
- Using descriptive dataframe names helps us distinguish between multiple dataframes so we won’t accidentally overwrite a dataframe or read from the wrong one.

## Use the `DataFrame.info()` method to find out more about a dataframe.

In [ ]:
solo_mag_data.info()

- This is a `DataFrame`
- 2880 rows
- Seven columns, all either integers or floating point numbers.
- We will talk later about null values, which are used to represent missing data.
- Uses 78.8 kilobytes of memory.

## The `DataFrame.columns` variable stores information about the dataframe’s columns.

- Note that this is data, _not_ a method. (It doesn’t have parentheses.)
  - Like `math.pi`.
  - So do not use `()` to try to call it.
- Called a member variable, or just member.

In [ ]:
print(solo_mag_data.columns)

## Use `DataFrame.T` to transpose a dataframe.

- Sometimes want to treat columns as rows and vice versa.
- Transpose (written `.T`) doesn’t copy the data, just changes the program’s view of it.
- Like `columns`, it is a member variable.

In [ ]:
print(solo_mag_data.T)

## Use `DataFrame.describe()` to get summary statistics about data.

`DataFrame.describe()` gets the summary statistics of only the columns that have numerical data. All other columns are ignored, unless you use the argument `include='all'`.

In [ ]:
print(solo_mag_data.describe())

## Key Points

- **Use the Pandas library to get basic statistics out of tabular data.**
- **Use `DataFrame.info` to find out more about a dataframe.**
- **The `DataFrame.columns` variable stores information about the dataframe’s columns.**
- **Use `DataFrame.T` to transpose a dataframe.**
- **Use `DataFrame.describe` to get summary statistics about data.**